In [14]:
import pandas as pd
from datetime import datetime, timedelta
import holidays

# URL do arquivo CSV no GitHub (link raw)
url = 'https://github.com/alexandreLoureiro/back/blob/master/estudo.csv'

In [11]:
# Carregar o DataFrame a partir do link raw do GitHub
df = pd.read_csv('estudo.csv', sep=';', encoding='latin1')

In [12]:
df

,NR_TRABALHO,Total_Horas,vinculo_atividade_planejamento,DEPE_atividade_planejamento,descricao_atividade_planejamento,tipo_atividade_planejamento,DT_INC_atividade_planejamento,DT_FIM_atividade_planejamento,horas_destinadas_trabalho
0,49320,2686,49823,8454,Comunicação de Resultados,Processo Corporativo,02DEC2024,04FEB2025,420
1,49320,2686,49812,8454,Pré-auditoria,Processo Corporativo,26FEB2024,07/jun/24,750
2,49320,2686,49822,8454,Execução,Processo Corporativo,10/jun/24,29/nov/24,1250
3,49320,2686,49824,8454,Supervisão,Processo Corporativo,26FEB2024,04FEB2025,266
4,49349,3010,49469,9647,Execução,Processo Corporativo,04/nov/24,27DEC2024,30
...,...,...,...,...,...,...,...,...,...
1551,54036,1929,54041,8454,Execução,Processo Corporativo,29SEP2025,06/nov/25,200
1552,54036,1929,54038,9889,Pré-auditoria,Processo Corporativo,25AUG2025,26SEP2025,525
1553,54036,1929,54040,9889,Execução,Processo Corporativo,29SEP2025,06/nov/25,609
1554,54036,1929,54042,9889,Comunicação de Resultados,Processo Corporativo,07/nov/25,21/nov/25,231


In [15]:
# Definir feriados do Brasil
br_holidays = holidays.Brazil()

In [26]:
# Certifique-se de que as colunas de data estão no formato datetime
def parse_dates(date_str):
    date_formats = ['%d%b%Y', '%d/%b/%y', '%d%b%y', '%d/%m/%y', '%d/%m/%Y', '%d%b%Y', '%d%b%y', '%d%b%Y']
    for fmt in date_formats:
        try:
            return pd.to_datetime(date_str, format=fmt, dayfirst=True)
        except (ValueError, TypeError):
            continue
    return pd.NaT


In [27]:
# Aplicar a função de análise de datas às colunas
df['DT_INC_atividade_planejamento'] = df['DT_INC_atividade_planejamento'].apply(parse_dates)
df['DT_FIM_atividade_planejamento'] = df['DT_FIM_atividade_planejamento'].apply(parse_dates)

# Verificar se as datas foram convertidas corretamente
if df['DT_INC_atividade_planejamento'].isnull().any() or df['DT_FIM_atividade_planejamento'].isnull().any():
    print("Atenção: Algumas datas não foram convertidas corretamente. Verifique os formatos de data.")


In [34]:
# Obter o ano mínimo e máximo das colunas de data
min_year = df['DT_INC_atividade_planejamento'].dt.year.min()
max_year = df['DT_FIM_atividade_planejamento'].dt.year.max()

# Criar a lista de anos dinamicamente
anos = list(range(min_year, max_year + 1))

In [35]:
anos

[2024, 2025, 2026]

In [29]:
# Para cada ano, calcule o número de dias úteis excluindo feriados
for ano in anos:
    inicio_ano = pd.Timestamp(f'{ano}-01-01')
    fim_ano = pd.Timestamp(f'{ano}-12-31')
    
    df[f'dias_uteis_{ano}'] = df.apply(
        lambda row: len(
            pd.bdate_range(
                start=max(row['DT_INC_atividade_planejamento'], inicio_ano), 
                end=min(row['DT_FIM_atividade_planejamento'], fim_ano),
                holidays=br_holidays
            )
        ) if pd.notnull(row['DT_INC_atividade_planejamento']) and pd.notnull(row['DT_FIM_atividade_planejamento']) and min(row['DT_FIM_atividade_planejamento'], fim_ano) >= max(row['DT_INC_atividade_planejamento'], inicio_ano) else 0,
        axis=1
    )

In [30]:
# Calcular o total de dias úteis no período total excluindo feriados
df['total_dias_uteis'] = df.apply(
    lambda row: len(
        pd.bdate_range(
            start=row['DT_INC_atividade_planejamento'], 
            end=row['DT_FIM_atividade_planejamento'],
            holidays=br_holidays
        )
    ) if pd.notnull(row['DT_INC_atividade_planejamento']) and pd.notnull(row['DT_FIM_atividade_planejamento']) else pd.NA,
    axis=1
)

In [31]:
# Evitar divisão por zero
df['total_dias_uteis'] = df['total_dias_uteis'].replace({0: pd.NA})

# Calcular horas por dia útil
df['horas_por_dia_util'] = df['horas_destinadas_trabalho'] / df['total_dias_uteis']

In [32]:
# Para cada ano, calcular as horas trabalhadas
for ano in anos:
    df[f'horas_{ano}'] = df[f'dias_uteis_{ano}'] * df['horas_por_dia_util']
    # Substituir valores nulos por zero
    df[f'horas_{ano}'] = df[f'horas_{ano}'].fillna(0)

# Selecionar as colunas para exibição, incluindo dias úteis por ano
colunas_resultado = ['NR_TRABALHO', 'DT_INC_atividade_planejamento', 'DT_FIM_atividade_planejamento', 'horas_destinadas_trabalho', 'total_dias_uteis'] + [f'dias_uteis_{ano}' for ano in anos] + [f'horas_{ano}' for ano in anos]

In [33]:
df[colunas_resultado]

,NR_TRABALHO,DT_INC_atividade_planejamento,DT_FIM_atividade_planejamento,horas_destinadas_trabalho,total_dias_uteis,dias_uteis_2024,dias_uteis_2025,dias_uteis_2026,horas_2024,horas_2025,horas_2026
0,49320,2024-12-02,2025-02-04,420,47,22,25,0,196.595745,223.404255,0.0
1,49320,2024-02-26,2024-06-07,750,75,75,0,0,750.000000,0.000000,0.0
2,49320,2024-06-10,2024-11-29,1250,125,125,0,0,1250.000000,0.000000,0.0
3,49320,2024-02-26,2025-02-04,266,247,222,25,0,239.076923,26.923077,0.0
4,49349,2024-11-04,2024-12-27,30,40,40,0,0,30.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1551,54036,2025-09-29,2025-11-06,200,29,0,29,0,0.000000,200.000000,0.0
1552,54036,2025-08-25,2025-09-26,525,25,0,25,0,0.000000,525.000000,0.0
1553,54036,2025-09-29,2025-11-06,609,29,0,29,0,0.000000,609.000000,0.0
1554,54036,2025-11-07,2025-11-21,231,11,0,11,0,0.000000,231.000000,0.0


In [38]:
df.query('horas_2026 > 0.0')

,NR_TRABALHO,Total_Horas,vinculo_atividade_planejamento,DEPE_atividade_planejamento,descricao_atividade_planejamento,tipo_atividade_planejamento,DT_INC_atividade_planejamento,DT_FIM_atividade_planejamento,horas_destinadas_trabalho,dias_uteis_2024,dias_uteis_2025,dias_uteis_2026,total_dias_uteis,horas_por_dia_util,horas_2024,horas_2025,horas_2026
311,51886,1560,53034,8960,Supervisão,Processo Corporativo,2025-10-01,2026-01-09,10,0,66,7,73,0.136986,0.0,9.041096,0.958904
314,51886,1560,51892,9604,Comunicação de Resultados,Processo Corporativo,2025-12-15,2026-01-09,200,0,13,7,20,10.000000,0.0,130.000000,70.000000
315,51886,1560,51893,9604,Supervisão,Processo Corporativo,2025-10-01,2026-01-09,80,0,66,7,73,1.095890,0.0,72.328767,7.671233
334,51956,1650,51959,8888,Execução,Processo Corporativo,2025-11-17,2026-02-06,900,0,33,27,60,15.000000,0.0,495.000000,405.000000
335,51956,1650,51960,8888,Comunicação de Resultados,Processo Corporativo,2026-02-09,2026-03-14,300,0,0,25,25,12.000000,0.0,0.000000,300.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,53726,2230,53744,9920,Supervisão,Processo Corporativo,2025-09-18,2026-01-09,60,0,75,7,82,0.731707,0.0,54.878049,5.121951
1476,53808,640,53915,9875,Supervisão,Processo Corporativo,2025-10-06,2026-01-15,40,0,63,11,74,0.540541,0.0,34.054054,5.945946
1477,53808,640,54057,9875,Comunicação de Resultados,Processo Corporativo,2026-01-02,2026-01-15,80,0,0,10,10,8.000000,0.0,0.000000,80.000000
1480,53809,770,53918,9875,Comunicação de Resultados,Processo Corporativo,2026-01-02,2026-01-22,80,0,0,15,15,5.333333,0.0,0.000000,80.000000
